# 1. Solution explanation. 
Justify how your algorithm works. Add a code listing and explain what this code does.


A naive way to solve the problem of finding the percentile is to concatenate arrays ***a*** and ***b*** and sort them and display the required percentile by the *original rank* ***k*** of the sorted array. But such an method will have high time complexity.  
Therefore, we will take advantage of the fact that the arrays are sorted and implement the percentile search algorithm using the ideas of the binary search algorithm.  


To search for a percentile, we need to find the k-th element corresponding to this percentile. To do this, we will select a range of length ***k*** in arrays ***a*** and ***b***, where ***k*** is ordinal rank for percentile ***P***:  
**k = (len(a) + len(b)) * P / 100**.


1. First, we take the same left sides of arrays ***a*** and ***b***, the total length of which will be ***k***.
This way each array will be split into 2 parts. And we get two pairs of parts - left and right.


2. Then we compare the left and right parts - if both left parts are less than the right ones, then our desired percentile is found, as a maximum from the right boundaries of the left parts of arrays ***a*** and ***b***.


3. To compare the left and right sides, we can use the knowledge that our arrays are sorted. We can compare only the borders of these parts - the right borders of the left parts (*left_a, left_b*) with the left borders of the right ones (*right_a, right_b*) (a bit confusing, but I think the inspectors know this algorithm well, and will skim this explanation).


4. If one of the right borders of the left side is larger than the left border of the left side, then we will increase the border of the second left side using a binary search approach. Then we will have to reduce the second left part so that the total length of the both left parts remains equal to the ordinal rank ***k***.


5. We will do this adjustment until the conditions for inequality of the boundaries are satisfied. And then we will stop our loop, since the desired percentile is found.

Thus, the complexity of our algorithm will be **O(log(n))**, where ***n*** is size of array ***a***.



In [1]:
import math

# The function checks if the index is out of array range:
# if the index isn't out of range, it returns element of array,
# if the index is out of range, it returns (-+)inf.
def _bound_validation(array, index):
    if (index >= 0) and (index <= len(array) - 1):
        return array[index]
    return math.inf * (-1 if index < 0 else 1)

#------------------------------------

# The function finds the k-th element in the two sorted arrays a and b
def _find_k(a, b, k):
    
    # determine the boundaries of the interval of the array a, within which the k-th element can be located
    high = min(len(a), k)
    low = max(0, k - len(b))
    
    # in the loop we are looking for a position for k-th element, until the interval for searching is empty
    while high >= low:
        # as a starting point we take half of the search interval
        # and we look for the remaining elements in b,
        # i.e. we select two intervals in arrays a and b so that their sum is equal to k    
        size_a = (high + low) // 2
        size_b = k - size_a 

        # divide the arrays a and b into 4 parts (2 pairs) 
        # in accordance with the selected boundaries of the intervals size_a and size_b
        left_a = _bound_validation(a, size_a - 1) 
        right_a = _bound_validation(a, size_a)
        left_b = _bound_validation(b, size_b - 1)
        right_b = _bound_validation(b, size_b)

        # k-th smallest element is taken from the maximum of the rightmost elements in the left partitions
        if (left_a <= right_b) and (left_b <= right_a):
            return max(left_a, left_b)
        
        # if the left bound of a is greater than the right bound of b, 
        # then we reduce the upper bound for partition a
        elif left_a > right_b:
            high = size_a - 1

        # if left_b > right_a, then we increase the lower bound for partition a 
        else:
            low = size_a + 1

#------------------------------------

# The function finds p-th percentile of sorted arrays a and b
def find_percentile(a, b, p):
    k = math.ceil((len(a) + len(b)) * p / 100)
    return _find_k(a, b, k)


#====================================


# some test code
if __name__ == "__main__":
    test_a, test_b, test_p = [1, 2, 7, 8, 10], [6, 12], 50
    # should print 7
    print(find_percentile(test_a, test_b, test_p))

    test_a, test_b, test_p = [1, 2, 7, 8], [6, 12], 50
    # should print 6
    print(find_percentile(test_a, test_b, test_p))

    test_a, test_b, test_p = [15, 20, 35, 40, 50], [], 30
    # should print 20
    print(find_percentile(test_a, test_b, test_p))

    test_a, test_b, test_p = [15, 20], [25, 40, 50], 40
    # should print 20
    print(find_percentile(test_a, test_b, test_p))

7
6
20
20


# 2. Time complexity. 


The time complexity of our algorithm is **O(log(n))**, where ***n*** is size of array ***a***.

At the entrance to the loop, we have the area of calibration **n/2**. If we do the calibration of the boundaries (the calibration conditions are true), then at the second iteration the area narrows to **n/4**, etc. Those, the time complexity of the algorithm is limited by the logarithm of the length of the array **a** (i. e. **O(log(n))**).


# 3. Space complexity.


We have space complexity **O(1)**, because we only store a limited number of variables to determine the boundaries of splitting arrays into 2 pairs.  
These are eight variables:  
1) *high*  
2) *low*   
3) *size_a*  
4) *size_b*  
5) *left_a*  
6) *right_a*  
7) *left_b*  
8) *right_b*  


The storage of these variables does not depend on the length of the input arrays, so the space complexity will be **O(1)**.

# 4. Correctness proof. 


Before the loop, we define the upper and lower bounds of the range for finding the k-th elements:  
1) If **k** is less than the size of array **a**, then we only need at most *k* elements in **a**'s left partition: **high = min(len(a), k)**;  
2) If all elements of array **b** are on the left side of **b**, then we need a minimum of *(k-len(b))* elements for the left part of array **a**. That is, the lower bound will be: **low = max(0, k - len(b))**.

###  4.1. All described loop invariants are correct and useful for finding final result:


In the loop of our algorithm, the invariant is the ordinal rank **k**, which remains unchanged throughout the entire operation of the loop, because the combined size of the left partitions should always remain the same (i.e. should be euqual to **k**):  
**k = size_a + size_b**


###     4.2. Correct initialization: 1 point


In the first iteration of the loop, we initialize the sizes of the left parts of arrays **a** and **b** so that the sum of the lengths of these parts is equal to the invariant **k**:   
**size_a = (high + low) // 2**  
**size_b = k - size_a**   

That is, the initialization of our invariant is true:  
**size_a + size_b = k**





###     4.3. Correct maintenance:

The loop contains the initialization of the boundaries of the left and right partitions of the arrays ***a*** and ***b***. The invariant ***k*** remains true.  


Further in the loop there is a bounds check conditions:

1. The first check shows that if the left boundaries of the parts of the arrays ***a*** and ***b*** are less than the right boundaries, then we don't need calibration, since all the elements of the left partitions will be less than the right ones, and their sum will be equal to the invariant ***k***. That is, we have found our percentile as the maximum of two boundaries of the left parts - we will return this maximum (*return max(left_a, left_b)*) and end the work of the loop.


2. If the first condition is false, then we get into the second condition, in which more or not the boundary of the left parts of array ***a*** is checked than the boundary of the right part of array ***b***. If the condition is true, then we carry out the calibration - we reduce the size of ***a*** by one (*high = size_a - 1*) and run the second iteration of loop.  


3. If the first two conditions are false, then the last option remains - when the boundary for the left part of array ***b*** is greater than the boundary for the right part of array ***a***. Then we increase the lower bound for the size of array ***a*** by one (*low = size_a + 1*) and go to the next iteration of the loop.

In all three conditions, the invariant ***k*** remains unchanged.


###  4.4 Correct termination:


The loop ends when the first condition for comparison boundaries is true. Then the maximum of the boundaries of the left partitions of arrays ***a*** and ***b*** is returned, and the left parts themselves contain ***k*** elements, i.e. the invariant doesn't change.

# 5. Tests

In [2]:
from random import seed, randint
from time import time

# -----------------------------------  

# The test checks the function find_percentile(a, b, p) for the correct calculation of the percentile

def test_find_percentile(a, b, p, correct_percentile):
    percentile = find_percentile(a, b, p)
    error_str = f"Test failed!\n Input:\n a:\t {a}\n b: \t{b}\n p:\t {p}\n\nResults:\n Percentile:\t {percentile}\n Correct percentile:\t {correct_percentile}"
    assert percentile == correct_percentile, error_str
#    print("Unit test passed!")

# -----------------------------------  

# The function runs unit tests

def run_unit_test():
    test_find_percentile([], [], 50, -math.inf)   
    test_find_percentile([1, 1], [1, 1, 1], 0, -math.inf)
    test_find_percentile([1, 1], [1, 1, 1], 1, 1)
    test_find_percentile([1, 1], [1, 1, 1], 100, 1)
    test_find_percentile([1, 2, 7, 8, 10], [6, 12], 50, 7)
    test_find_percentile([1, 2, 7, 8], [6, 12], 50, 6)
    test_find_percentile([15, 20, 35, 40, 50], [], 30, 20)
    test_find_percentile([15, 20], [25, 40, 50], 40, 20)
    test_find_percentile([12, 20, 25, 25, 26], [20, 25, 26, 26, 27, 27, 40, 40], 73, 27)
    print("All unit tests passed!")

# -----------------------------------  

# A simple implementation to find the p-percentile in the concatenation of two sorted arrays a and b

def simple_method_find_percentile(a, b, p):
    merged_array = sorted(a + b)
    k = math.ceil(len(merged_array) * p / 100)
    return merged_array[k-1]

# -----------------------------------  

#The function runs many random tests

def run_stress_test(count_tests=1000, max_val=10**10, max_len=1000):
    seed(1000)
    for _ in range(count_tests):
        a = sorted([randint(0, max_val) for _ in range(randint(1, max_len))])
        b = sorted([randint(0, max_val) for _ in range(randint(0, max_len))])
        p = randint(1, 100)
        correct_percentile = simple_method_find_percentile(a, b, p)
        test_find_percentile(a, b, p, correct_percentile)
    print("All stress tests passed!")
        
# -----------------------------------  

#The function runs max test: a random test with large arrays a and b.
#The find_percentile works 0.8178 seconds on the max test

def run_max_test(max_val=10**10, max_len=10**6):
    seed(100)
    a = sorted([randint(0, max_val) for _ in range(max_len)])
    b = sorted([randint(0, max_val) for _ in range(max_len)])
    p = 100
    correct_percentile = simple_method_find_percentile(a, b, p)

    start = time()
    test_find_percentile(a, b, p, correct_percentile)
    end = time()

    print("Max test passed!")
    print(f"Execution time of max test (in seconds): {round((end - start), 4)}")

# -----------------------------------  

def main():
    run_unit_test()
    run_stress_test()
    run_max_test()
    
if __name__ == '__main__':
    main()

All unit tests passed!
All stress tests passed!
Max test passed!
Execution time of max test (in seconds): 0.8603



________________________________

~ *Prepared by the student: Afanasiev Sergey* ~